# Set Device

I set up a variable, checking if there is a GPU avaliable to run the network on. 

In [3]:
device = torch.device("cudo" if torch.cuda.is_available() else "cpu")

In [2]:

import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np

from os.path import isfile
from data_handling.image_processing import PrepareData

In [14]:
image_size = 128
credentials_location = ".gitignore/credentials_for_marketplace.yml"
images_folder = "data/cleaned_images_" + str(image_size)
X_array_folder = "data/X_for_image_"+ str(image_size) + ".npy"
y_array_folder = "data/y_for_image_"+ str(image_size)+".npy"



if isfile(X_array_folder) and isfile(y_array_folder): 
    X = np.load(X_array_folder)
    y = np.load(y_array_folder)
else:
    pipeline = PrepareData()
    product_df, image_df = pipeline.retrieve_dataframes(credentials_location)
    X, y = pipeline.form_arrays(images_folder, image_size)
    pipeline.save_to_files(X_array_folder, y_array_folder)


data/images_table.json  is already downloaded, skipping
data/products_table.json  is already downloaded, skipping


In [15]:
class ProductImageCategoryDataset(Dataset):
    def __init__(self, X, y, img_side_length):
        super().__init__()

        self.X = X
        self.y = y
        self.img_side_length = img_side_length

        # self.X = np.load("data/X_for_image_128.npy")
        # self.y = np.load("data/y_for_image_128.npy")

        # self.X = pd.read_pickle('image_model_X.pkl')
        # self.y = pd.read_pickle('image_model_y.pkl')
        
        assert len(self.X) == len(self.y)

    def __getitem__(self, index):
        features = self.X[index]
        label = self.y[index]
        print(index)
        features = torch.tensor(features)
        features = features.reshape(3, self.img_side_length, self.img_side_length)

        label = int(label)

        return (features, label)

    def __len__(self):
        return len(self.X)



In [21]:

dataset = ProductImageCategoryDataset(X,y, image_size)


In [22]:
print(len(dataset))

12604


In [23]:

train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [26]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(3, 8, 9),
            torch.nn.Flatten(),  # flatten
            # simplify further with linear layers
            torch.nn.Linear(115200, 14),
            torch.nn.Softmax()  # turn into probabilities
        )

    def forward(self, features):
        return self.layers(features)



In [28]:

model = CNN()
optimiser = torch.optim.Adam(model.parameters(), lr = 0.001)

for batch_index, batch in enumerate(train_loader):
    # print(batch)
    features, labels = batch
    prediction = model(features.float())
    
    # calculate the loss
    loss = F.cross_entropy(prediction, labels)
    print("Batch: ", batch_index, "\t Loss: ", loss.item())

    # calculate the gradients 
    loss.backward()

    #take an optimisation step
    optimiser.step()

    optimiser.zero_grad()
    




9910
5589
377
8804
Batch:  0 	 Loss:  2.633509635925293
3577
12603
3574
9645
Batch:  1 	 Loss:  2.658625602722168


/home/amy/miniconda3/envs/rec_ranking/lib/python3.10/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


2636
5778
4999
3687
Batch:  2 	 Loss:  2.75482439994812
993
2713
1776
1196
Batch:  3 	 Loss:  2.75482439994812
8880
12527
1617
7143
Batch:  4 	 Loss:  2.75482439994812
7211
10543
4537
11393
Batch:  5 	 Loss:  2.50482439994812
10591
1053
5375
2640
Batch:  6 	 Loss:  2.50482439994812
5762
12082
1837
11862
Batch:  7 	 Loss:  2.75482439994812
1522
6599
11403
6985
Batch:  8 	 Loss:  2.75482439994812
6613
5548
5969
4584
Batch:  9 	 Loss:  2.75482439994812
8420
8607
523
2573
Batch:  10 	 Loss:  2.75482439994812
6968
8877
6786
1411
Batch:  11 	 Loss:  2.75482439994812
10346
7885
2891
5098
Batch:  12 	 Loss:  2.50482439994812
709
2700
5309
472
Batch:  13 	 Loss:  2.75482439994812
1415
2571
9074
11706
Batch:  14 	 Loss:  2.75482439994812
11414
7137
9752
5452
Batch:  15 	 Loss:  2.50482439994812
68
920
12074
12243
Batch:  16 	 Loss:  2.75482439994812
6369
9075
8480
11676
Batch:  17 	 Loss:  2.75482439994812
8818
9372
9551
6251
Batch:  18 	 Loss:  2.50482439994812
11691
10456
11898
6368
Batch:  19